**Table of contents**<a id='toc0_'></a>    
- [Importar Librerias](#toc1_)    
- [Extraer data de los jugadores](#toc2_)    
- [Transformar el dataset final](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

Ej: https://www.sofascore.com/api/v1/event/14115270/lineups

In [1]:
import json

# Carga segura del JSON
try:
    with open('../data/raw/lineups_event14249737.json', 'r') as f:
        data = json.load(f)
except (FileNotFoundError, json.JSONDecodeError) as e:
    print(f"Error cargando el JSON: {e}")
    data = {}

# Lista de todos los jugadores
players = data.get("home", {}).get("players", []) + data.get("away", {}).get("players", [])

# Extraer los dos teamId en orden de aparición
team_ids = []
for p in players:
    tid = p.get("teamId")
    if tid is not None and tid not in team_ids:
        team_ids.append(tid)
    if len(team_ids) == 2:
        break

# Lista para almacenar las 22 URLs
urls = []

# Para cada teamId, agrega sus primeros 11 jugadores a la lista
for tid in team_ids:
    count = 0
    for entry in players:
        if entry.get("teamId") != tid:
            continue
        player = entry.get("player", {})
        slug = player.get("slug")
        pid  = player.get("id")
        if slug and pid:
            urls.append(f"https://www.sofascore.com/es/football/player/{slug}/{pid}")
            count += 1
        if count == 11:
            break

# Imprimir las 22 URLs en orden
for url in urls:
    print(url)

https://www.sofascore.com/es/football/player/james-aguirre/829877
https://www.sofascore.com/es/football/player/juan-david-cuesta/1188964
https://www.sofascore.com/es/football/player/mateo-garcia/950544
https://www.sofascore.com/es/football/player/jaider-riquett/924615
https://www.sofascore.com/es/football/player/palacios-luis/1130300
https://www.sofascore.com/es/football/player/cuesta-rodriguez-kevin-andres/1067105
https://www.sofascore.com/es/football/player/michael-barrios/791545
https://www.sofascore.com/es/football/player/luis-sanchez/963525
https://www.sofascore.com/es/football/player/alejandro-garcia/912291
https://www.sofascore.com/es/football/player/mateo-zuleta/1651712
https://www.sofascore.com/es/football/player/dayro-moreno/31715
https://www.sofascore.com/es/football/player/hernan-galindez/588618
https://www.sofascore.com/es/football/player/tomas-guidara/871519
https://www.sofascore.com/es/football/player/fabio-pereyra/943995
https://www.sofascore.com/es/football/player/nehu

In [2]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    WebDriverException
)
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc
# Lista de URLs de jugadores (ejemplo: 22 URLs)
player_urls = urls

# User‐agents variados para rotación
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
]


def create_stealth_driver():
    # options = uc.ChromeOptions()
    # # Rotar user-agent
    # ua = random.choice(USER_AGENTS)
    # options.add_argument(f"--user-agent={ua}")
    # # Desactivar flags de detección
    # options.add_argument("--disable-blink-features=AutomationControlled")
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")
    # # Ventana aleatoria
    # width = random.randint(1024, 1920)
    # height = random.randint(768, 1080)
    # options.add_argument(f"--window-size={width},{height}")

    # driver = uc.Chrome(options=options)
    # driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    #     'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
    # })
    # return driver

    options = uc.ChromeOptions()
    ua = random.choice(USER_AGENTS)
    options.add_argument(f"--user-agent={ua}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    width = random.randint(1024, 1920)
    height = random.randint(768, 1080)
    options.add_argument(f"--window-size={width},{height}")

    # <— fuerza la versión 138
    driver = uc.Chrome(version_main=138, options=options)
    driver.execute_cdp_cmd(
        'Page.addScriptToEvaluateOnNewDocument',
        {'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'}
    )
    return driver


def scrape_player_stats(driver, player_url):
    driver.get(player_url)
    # Espera inicial aleatoria
    time.sleep(random.uniform(2, 4))

    # Datos básicos
    try:
        name_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h2")))
        player_name = name_elem.text.strip()
    except Exception:
        player_name = None

    try:
        team_elem = driver.find_element(By.XPATH, "(//div[contains(@class,'flex-d_column') and contains(@class,'jc_center')]//span[contains(@class,'textStyle_body')])[1]")
        team_name = team_elem.text.strip()
    except Exception:
        team_name = None

    try:
        pos_elem = driver.find_element(By.CSS_SELECTOR, "div.Box.oWZdE .Text.beCNLk")
        position = pos_elem.text.strip()
    except Exception:
        position = None

    # Click modal
    try:
        triggers = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.Box.gRFdPz")))
        trigger = triggers[-1]
        driver.execute_script("arguments[0].scrollIntoView({block:'center'})", trigger)
        time.sleep(random.uniform(0.5, 1.0))
        ActionChains(driver).move_to_element(trigger).click().perform()
    except Exception:
        pass

    # Extraer estadísticas
    stats = {}
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.ov-x_clip")))
        time.sleep(random.uniform(2, 3))
        # Ocultar overlays
        driver.execute_script("document.querySelectorAll('[id^=\"div-gpt-ad\"], .overlay, .banner').forEach(el => el.style.display='none');")
        labels = driver.find_elements(By.CSS_SELECTOR, "span[class*='ov-x_clip']")
        for label in labels:
            if not label.is_displayed():
                continue
            try:
                value = driver.execute_script("return arguments[0].nextElementSibling", label)
                text_val = value.text.strip() if value else ""
            except:
                continue
            ltxt = label.text.strip()
            if ltxt and text_val:
                stats[ltxt] = text_val
    except Exception:
        pass

    # Construir registro
    record = {
        "Nombre": player_name,
        "Equipo": team_name,
        "Posición": position,
        **stats
    }
    return record


def main():
    driver = create_stealth_driver()
    all_records = []
    for url in player_urls:
        try:
            rec = scrape_player_stats(driver, url)
            all_records.append(rec)
            # Espera corta antes del siguiente jugador
            time.sleep(random.uniform(1, 2))
        except Exception:
            continue

    driver.quit()

    # Convertir a DataFrame
    df = pd.DataFrame(all_records)
    print(df)
    # Guardar a CSV
    df.to_csv("jugadores_stats.csv", index=False)

    return df

if __name__ == "__main__":
    df_final = main()


                           Nombre       Equipo Posición Minutos jugados  \
0                   James Aguirre  Once Caldas        G             90'   
1               Juan David Cuesta  Once Caldas        D             90'   
2                    Mateo García  Once Caldas        M             31'   
3                  Jaider Riquett  Once Caldas        D             90'   
4                   Luis Palacios  Once Caldas        M             24'   
5   Kevin Andres Cuesta Rodriguez  Once Caldas        D             90'   
6                 Michael Barrios  Once Caldas        M             66'   
7                    Luis Sánchez  Once Caldas        M             66'   
8                Alejandro García  Once Caldas        M             59'   
9                    Mateo Zuleta  Once Caldas        M             59'   
10                   Dayro Moreno  Once Caldas        F             90'   
11                Hernán Galíndez      Huracán        G             90'   
12                  Tomás

In [3]:
df_final.loc[[17,9], 'Equipo'] = 'Huracán'
#df_final.loc[[15, 19, 20, 21], 'Equipo'] = 'Deportivo Cali'
df_final.fillna(0, inplace=True)
df_final

,Nombre,Equipo,Posición,Minutos jugados,Asistencias,Paradas,Con los puños,Salidas (comp.),Despejes por alto,Paradas a tiros desde dentro del área,...,Tiros a puerta,Tiros fuera,Regates intentados (completados),Goles,Posesiones perdidas,Ocas. claras creadas,Errores que llevan a disparo,Ocas. claras falladas,Fueras de juego,Asistencia esperada (xA)
0,James Aguirre,Once Caldas,G,90',0,4,0,0 (0),0,3,...,0,0,0 (0),0,0,0,0,0,0,0
1,Juan David Cuesta,Once Caldas,D,90',0,0,0,0,0,0,...,1,1,0 (0),0,9,0,0,0,0,0
2,Mateo García,Once Caldas,M,31',0,0,0,0,0,0,...,1,0,0 (0),0,1,0,0,0,0,0
3,Jaider Riquett,Once Caldas,D,90',0,0,0,0,0,0,...,0,0,0 (0),0,5,0,0,0,0,0
4,Luis Palacios,Once Caldas,M,24',1,0,0,0,0,0,...,0,0,0 (0),0,3,0,0,0,0,0
5,Kevin Andres Cuesta Rodriguez,Once Caldas,D,90',0,0,0,0,0,0,...,0,0,0 (0),0,4,0,0,0,0,0
6,Michael Barrios,Once Caldas,M,66',1,0,0,0,0,0,...,2,0,2 (0),1,10,1,0,0,0,0
7,Luis Sánchez,Once Caldas,M,66',0,0,0,0,0,0,...,0,1,1 (1),0,11,0,1,0,0,0
8,Alejandro García,Once Caldas,M,59',0,0,0,0,0,0,...,0,0,0 (0),0,12,0,0,0,0,0
9,Mateo Zuleta,Huracán,M,59',0,0,0,0,0,0,...,1,0,4 (2),1,9,0,0,0,0,0


In [4]:
df_final.to_csv("jugadores_stats.csv", index=False)

In [5]:
#Minutos Jugados
df_final["Minutos jugados"] = df_final["Minutos jugados"].str.rstrip("'")

#Pases Precisos 
df_final[['Pases Acertados', 'Total Pases', 'Efectividad Pases (%)']] = df_final['Pases precisos'].str.extract(r'(\d+)/(\d+)\s+\((\d+)%\)')

#Centros
df_final[['Total Centros', 'Centros Acertados']] = df_final['Centros (acertados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Centros'] = (
    df_final.apply(lambda row: (row['Centros Acertados'] / row['Total Centros'] * 100) if row['Total Centros'] != 0 else 0, axis=1).round(2)
)

#pases largos
df_final[['Total Pases Largos', 'Pases Largos Acertados']] = df_final['Pases largos (acertados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Pases Largos'] = (
    df_final.apply(lambda row: (row['Pases Largos Acertados'] / row['Total Pases Largos'] * 100) if row['Total Pases Largos'] != 0 else 0, axis=1).round(2)
)

#Duelos en el suelo
df_final[['Total Duelos el Suelo', 'Duelos Ganados en el Suelo']] = df_final['Duelos en el suelo (ganados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Duelos en el Suelo'] = (
    df_final.apply(lambda row: (row['Duelos Ganados en el Suelo'] / row['Total Duelos el Suelo'] * 100) if row['Total Duelos el Suelo'] != 0 else 0, axis=1).round(2)
)

#Duelos aéreos
df_final[['Total Duelos Aéreos', 'Duelos Ganados en el Aire']] = df_final['Duelos aéreos (ganados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Duelos Aéreos'] = (
    df_final.apply(lambda row: (row['Duelos Ganados en el Aire'] / row['Total Duelos Aéreos'] * 100) if row['Total Duelos Aéreos'] != 0 else 0, axis=1).round(2)
)

#Regates
df_final[['Total Regates', 'Regates Completados']] = df_final['Regates intentados (completados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Regates'] = (
    df_final.apply(lambda row: (row['Regates Completados'] / row['Total Regates'] * 100) if row['Total Regates'] != 0 else 0, axis=1).round(2)
)



In [6]:
# Columnas tipo texto (al inicio)
columnas_texto = ['Nombre', 'Equipo', 'Posición']

# Columnas numéricas corregidas
columnas_numericas = [
    'Minutos jugados', 'Asistencias', 'Toques', 'Total Pases', 'Pases Acertados',
    'Efectividad Pases (%)', 'Pases clave', 'Total Centros', 'Centros Acertados',
    '% Efectividad Centros', 'Total Pases Largos', 'Pases Largos Acertados',
    '% Efectividad Pases Largos', 'Despejes', 'Tiros bloqueados', 'Intercepciones',
    'Tackles totales', 'Regateado', 'Total Duelos el Suelo', 'Duelos Ganados en el Suelo',
    '% Efectividad Duelos en el Suelo', 'Total Duelos Aéreos', 'Duelos Ganados en el Aire',
    '% Efectividad Duelos Aéreos', 'Faltas', 'Tiros a puerta', 'Tiros fuera',
    'Total Regates', 'Regates Completados', '% Efectividad Regates',
    'Goles', 'Posesiones perdidas', 'Ocas. claras creadas', 'Fueras de juego'
]

# Concatenar texto + numéricas ya convertidas
df_vf = pd.concat([
    df_final[columnas_texto],  # columnas de texto
    df_final[columnas_numericas].apply(pd.to_numeric, errors="coerce")  # columnas numéricas convertidas
], axis=1)

# Reemplazar NaN por 0
df_vf = df_vf.fillna(0)

In [7]:
df_vf

,Nombre,Equipo,Posición,Minutos jugados,Asistencias,Toques,Total Pases,Pases Acertados,Efectividad Pases (%),Pases clave,...,Faltas,Tiros a puerta,Tiros fuera,Total Regates,Regates Completados,% Efectividad Regates,Goles,Posesiones perdidas,Ocas. claras creadas,Fueras de juego
0,James Aguirre,Once Caldas,G,90,0,46,36,35,97,0,...,0,0,0,0,0,0.00,0,0,0,0
1,Juan David Cuesta,Once Caldas,D,90,0,65,49,45,92,0,...,0,1,1,0,0,0.00,0,9,0,0
2,Mateo García,Once Caldas,M,31,0,28,24,23,96,1,...,1,1,0,0,0,0.00,0,1,0,0
3,Jaider Riquett,Once Caldas,D,90,0,56,45,41,91,0,...,0,0,0,0,0,0.00,0,5,0,0
4,Luis Palacios,Once Caldas,M,24,1,16,13,11,85,4,...,1,0,0,0,0,0.00,0,3,0,0
5,Kevin Andres Cuesta Rodriguez,Once Caldas,D,90,0,50,37,33,89,0,...,2,0,0,0,0,0.00,0,4,0,0
6,Michael Barrios,Once Caldas,M,66,1,42,28,24,86,3,...,2,2,0,2,0,0.00,1,10,1,0
7,Luis Sánchez,Once Caldas,M,66,0,64,48,42,88,1,...,2,0,1,1,1,100.00,0,11,0,0
8,Alejandro García,Once Caldas,M,59,0,50,42,34,81,0,...,0,0,0,0,0,0.00,0,12,0,0
9,Mateo Zuleta,Huracán,M,59,0,50,33,30,91,0,...,1,1,0,4,2,50.00,1,9,0,0


In [8]:
ofensivas = [
    'Minutos jugados', 'Toques', 'Total Pases', 'Pases Acertados', 'Efectividad Pases (%)',
    'Asistencias', 'Pases clave', 'Total Centros', 'Centros Acertados',
    '% Efectividad Centros', 'Total Pases Largos', 'Pases Largos Acertados',
    '% Efectividad Pases Largos', 'Tiros a puerta', 'Tiros fuera',
    'Total Regates', 'Regates Completados', '% Efectividad Regates',
    'Goles', 'Ocas. claras creadas', 'Fueras de juego'
]

defensivas = [
    'Despejes', 'Tiros bloqueados', 'Intercepciones', 'Tackles totales',
    'Regateado', 'Total Duelos el Suelo', 'Duelos Ganados en el Suelo',
    '% Efectividad Duelos en el Suelo', 'Total Duelos Aéreos',
    'Duelos Ganados en el Aire', '% Efectividad Duelos Aéreos',
    'Faltas', 'Posesiones perdidas'
]

# 1) Limpiar y convertir a numérico
for col in ofensivas + defensivas:
    if col in df_final.columns:
        df_final[col] = (
            df_final[col]
            .astype(str)
            .str.replace(r'[%,]', '', regex=True)
        )
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce').fillna(0)

# 2) Agrupar por equipo
ofensiva_por_equipo = df_final.groupby('Equipo')[ofensivas].sum().reset_index()
defensiva_por_equipo = df_final.groupby('Equipo')[defensivas].sum().reset_index()

# 3) Función para resaltar máximo y mínimo
def resaltar_max_min(s):
    if s.name == 'Equipo':
        return [''] * len(s)
    es_num = pd.to_numeric(s, errors='coerce')
    max_val = es_num.max()
    min_val = es_num.min()
    return [
        'background-color: green; color: black' if v == max_val
        else 'background-color: red; color: black' if v == min_val
        else 'color: black'
        for v in es_num
    ]

# Aplicar estilo con formato sin decimales
ofensiva_styled = (
    ofensiva_por_equipo
    .style
    .apply(resaltar_max_min, axis=0)
    .format(precision=0)
)

defensiva_styled = (
    defensiva_por_equipo
    .style
    .apply(resaltar_max_min, axis=0)
    .format(precision=0)
)

In [9]:
defensiva_styled

,Equipo,Despejes,Tiros bloqueados,Intercepciones,Tackles totales,Regateado,Total Duelos el Suelo,Duelos Ganados en el Suelo,% Efectividad Duelos en el Suelo,Total Duelos Aéreos,Duelos Ganados en el Aire,% Efectividad Duelos Aéreos,Faltas,Posesiones perdidas
0,Huracán,29,3,10,14,8,69,34,518,23,14,350,15,124
1,Once Caldas,9,2,2,10,2,32,18,537,3,2,150,8,60


In [10]:
ofensiva_styled

,Equipo,Minutos jugados,Toques,Total Pases,Pases Acertados,Efectividad Pases (%),Asistencias,Pases clave,Total Centros,Centros Acertados,% Efectividad Centros,Total Pases Largos,Pases Largos Acertados,% Efectividad Pases Largos,Tiros a puerta,Tiros fuera,Total Regates,Regates Completados,% Efectividad Regates,Goles,Ocas. claras creadas,Fueras de juego
0,Huracán,869,449,255,173,819,0,5,19,3,67,44,12,448,2,2,21,10,267,1,0,1
1,Once Caldas,696,445,344,307,891,3,12,9,2,50,44,34,603,6,3,4,2,200,1,2,3
